In [46]:
import re
import pandas as pd
import os
dataset_annotation_path = r"A:\AML project\dataset\IEMOCAP\nas\database\IEMOCAP_full_release\Session1\dialog\EmoEvaluation"
dataset_transcription_path = r"A:\AML project\dataset\IEMOCAP\nas\database\IEMOCAP_full_release\Session1\dialog\transcriptions"

In [60]:
annotation_files = [file for file in os.listdir(dataset_annotation_path) if file.endswith(".txt")]
'''geting the content of the files as [sentences and labels] and saving them to csv file per every diglouge'''
useful_regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
for file in annotation_files:
    annotation_dict = {}
    training_data = []
    path = os.path.join(dataset_annotation_path, file)
    f = open(path,'r')
    content = f.read()
    data = re.findall(useful_regex, content)
    for line in data[1:]:
        d = line.split()
        annotation_dict[d[3]] = d[4]
    f.close()
    #getting the utterance from transcription folder
    path = os.path.join(dataset_transcription_path,file)
    f = open(path,'r')
    content = f.readlines()
    for line in content:
        line = line.split(":")
        U_id = line[0].split()
        try:
            training_data.append([U_id[0],line[1].strip('\n'),annotation_dict[U_id[0]]])
        except:
            # training_data.append([U_id[0],line[1].strip('\n'),""])
            print("error")
    #create df for training data and save it as csv
    df = pd.DataFrame(training_data,columns=["Uid","Utterances","Labels"])
    file = file.strip(".txt")
    df.to_csv(file+".csv",index=False)

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
